In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sps
from filtered_point_process.model import Model
from filtered_point_process.point_processes.filtered_point_process import FilteredPointProcess
import os

# ─── PLOT STYLING ───────────────────────────────────────────────────
plt.rcParams.update({
    'font.size':          16,
    'axes.titlesize':     16,
    'axes.labelsize':     16,
    'xtick.labelsize':    14,
    'ytick.labelsize':    14,
    'legend.fontsize':    16,
    'figure.titlesize':   16,
})

# ─── PARAMETERS ────────────────────────────────────────────────────
FS           = 1000.0    # sampling rate (Hz)
DURATION     = 5.0       # total simulation time (s)
SEED         = 17
WIN_DUR      = 0.2       # 0.2 s windows
WIN_STEP     = 0.2       # 0.2 s hop
SHORT_TIMES  = [0.3, 4.3]  # window‐centers for B & C
FMIN, FMAX   = 0.1, 300.0  # full PSD range
ymin, ymax   = 1e-12, 1e-1
colors       = ['purple','pink']  # B → purple, C → pink

# ─── SIMULATION & FILTERING ────────────────────────────────────────
model_params = {
    "peak_height":      [50000],
    "center_frequency": [1.0],
    "peak_width":       [0.5],
    "lambda_0":         [1000.0],
}
sim_params = {
    "fs":       FS,
    "T":        DURATION,
    "simulate": True,
    "Nsims":    1,
    "seed":     SEED
}

model = Model("gaussian", model_params, sim_params)
fpp   = FilteredPointProcess(model=model, filters={"f1":"GABA","f2":"1/f"})
fpp.apply_filter_sequences([["f1","f2"]])
ts    = fpp.final_time_series_per_process[0]

# ─── FULL-TRACE HANN-TAPER PSD ────────────────────────────────────
freqs_emp, psd_emp = sps.periodogram(ts, fs=FS, window='hann', scaling='density')
mask_emp       = (freqs_emp >= FMIN) & (freqs_emp <= FMAX)
fr_emp         = freqs_emp[mask_emp]
psd_emp_full   = psd_emp[mask_emp]

# ─── FULL-TRACE THEORETICAL SPECTRUM ──────────────────────────────
spec           = fpp.get_final_spectrum()
freqs_th       = fpp.model.frequencies
psd_th         = spec["final_total_spectrum"]
mask_th        = (freqs_th >= FMIN) & (freqs_th <= FMAX)
fr_th          = freqs_th[mask_th]
psd_th_full    = psd_th[mask_th]

# ─── WINDOWED HANN-TAPER SPECTROGRAM ──────────────────────────────
f_win, t_win, Sxx = sps.spectrogram(
    ts,
    fs=FS,
    window='hann',
    nperseg=int(WIN_DUR*FS),
    noverlap=int((WIN_DUR-WIN_STEP)*FS),
    scaling='density',
    mode='psd'
)
mask_win_full = (f_win >= FMIN) & (f_win <= FMAX)
fr_win        = f_win[mask_win_full]
psd_win_lin_m = Sxx[mask_win_full, :].T    # (n_windows × n_freq)
time_edges    = np.arange(psd_win_lin_m.shape[0] + 1) * WIN_STEP

# ─── INSTANTANEOUS RATE ───────────────────────────────────────────
time_axis  = fpp.model.time_axis
lam        = np.real(fpp.model.cif.cif_realization)
n_win      = psd_win_lin_m.shape[0]
win_centers = np.arange(n_win)*WIN_STEP + WIN_DUR/2
win_idx     = [np.argmin(np.abs(win_centers - t)) for t in SHORT_TIMES]
lam_idx     = [np.argmin(np.abs(time_axis   - t)) for t in SHORT_TIMES]

# ─── FILTER-ONLY SPECTRUM ─────────────────────────────────────────
H1            = fpp.filter_instances['f1'].kernel_spectrum
H2            = fpp.filter_instances['f2'].kernel_spectrum
Htot_emp_full = H1 * H2
# on full PSD grid:
Htot_emp = np.interp(fr_emp, freqs_th, Htot_emp_full)
# on windowed grid:
Htot_win = np.interp(fr_win, freqs_th, Htot_emp_full)

# ─── THEORETICAL SPECTROGRAM (dB) ─────────────────────────────────
spec_th      = np.outer(lam, Htot_emp)
spec_th_dB   = 10 * np.log10(spec_th)
vmin2, vmax2 = np.percentile(spec_th_dB, [1,99])

# ─── EMPIRICAL SPECTROGRAM (dB) ──────────────────────────────────
psd_win_dB   = 10 * np.log10(psd_win_lin_m)
vmin3, vmax3 = np.percentile(psd_win_dB, [1,99])

# ─── MULTI-PANEL PLOT ─────────────────────────────────────────────
fig = plt.figure(figsize=(14,18))
gs  = fig.add_gridspec(4, 3,
                       height_ratios=[1, 0.5, 1, 1.2],
                       hspace=0.4)

# ─ Row 1A: Full PSD vs Theory ─────────────────────────────────────
ax1 = fig.add_subplot(gs[0,0])
ax1.loglog(fr_emp,   psd_emp_full, color='lightgrey', lw=1)
ax1.loglog(fr_th,    psd_th_full,  color='black',     lw=2)
ax1.set(xscale='log', yscale='log',
        xlim=(FMIN, FMAX), ylim=(ymin, ymax),
        title='Long Time Scale')
ax1.set_ylabel('Power')
ax1.set_xlabel('Frequency (log₁₀ Hz)')

# ─ Row 1B & 1C: Snapshots at SHORT_TIMES ─────────────────────────
for i, (t, widx, li) in enumerate(zip(SHORT_TIMES, win_idx, lam_idx), start=1):
    ax = fig.add_subplot(gs[0, i], sharex=ax1)
    mask_short    = fr_win >= 10
    freqs_short   = fr_win[mask_short]
    emp_short     = psd_win_lin_m[widx][mask_short]
    theo_full_win = lam[li] * Htot_win
    theo_short    = theo_full_win[mask_short]

    ax.loglog(freqs_short, emp_short,  color='lightgrey', lw=1)
    ax.loglog(freqs_short, theo_short, color=colors[i-1],     lw=2)
    ax.set(xscale='log', yscale='log',
           xlim=(FMIN, FMAX), ylim=(ymin, 1e-4),
           title=f"Short Time Scale ({t-WIN_DUR/2:.1f}–{t+WIN_DUR/2:.1f} s)")
    ax.set_xlabel('Frequency (log₁₀ Hz)')

# ─ Row 2: λ(t) over first 5 s ───────────────────────────────────
axλ = fig.add_subplot(gs[1, :])
axλ.plot(time_axis, lam, lw=1.5, color='black')
axλ.set(xlim=(0, DURATION), ylabel='λ(t)', title='Instantaneous Rate')
for t,c in zip(SHORT_TIMES, colors):
    axλ.axvline(t, color=c, lw=2)
axλ.axvspan(SHORT_TIMES[0]-WIN_DUR/2, SHORT_TIMES[0]+WIN_DUR/2,
            color='lightgrey', alpha=0.2)
axλ.axvspan(SHORT_TIMES[1]-WIN_DUR/2, SHORT_TIMES[1]+WIN_DUR/2,
            color='lightgrey', alpha=0.2)

# ─ Row 3: Theoretical Spectrogram ─────────────────────────────────
mask_spec        = fr_emp >= 10
fr_spec          = fr_emp[mask_spec]
spec_th_dB_mask  = spec_th_dB[:, mask_spec]

ax2 = fig.add_subplot(gs[2, :])
ax2.set_yscale('log')
pcm2 = ax2.imshow(
    spec_th_dB_mask.T,                    # data array (rows → freqs, cols → times)
    origin='lower',                       # put lowest freq at bottom
    aspect='auto',                        # allow independent axis scaling
    extent=[
        time_edges[0], time_edges[-1],    # t_min, t_max
        fr_spec[0],    fr_spec[-1]        # f_min, f_max
    ],
    cmap='viridis',
    vmin=vmin2,
    vmax=vmax2
)

for t,c in zip(SHORT_TIMES, colors):
    ax2.axvline(t, color=c, lw=2)
ax2.set(xlabel='Time (s)', ylabel='Frequency (Hz)',
        title='Theoretical Spectrogram (dB)',
        xlim=(0, DURATION), ylim=(10, FMAX))

# ─ Row 4: Empirical Spectrogram ──────────────────────────────────
mask_spec_emp      = fr_win >= 10
fr_spec_emp        = fr_win[mask_spec_emp]
psd_win_dB_masked  = psd_win_dB[:, mask_spec_emp]

ax3 = fig.add_subplot(gs[3, :])
ax3.set_yscale('log')
# pcm3 replacement
pcm3 = ax3.imshow(
    psd_win_dB_masked.T,                 
    origin='lower',
    aspect='auto',
    extent=[
        time_edges[0],     time_edges[-1],     
        fr_spec_emp[0],    fr_spec_emp[-1]     
    ],
    cmap='viridis',
    vmin=vmin3,
    vmax=vmax3
)

for t,c in zip(SHORT_TIMES, colors):
    ax3.axvline(t, color=c, lw=2)
ax3.set(xlabel='Time (s)', ylabel='Frequency (Hz)',
        title='Empirical Spectrogram (dB)',
        xlim=(0, DURATION), ylim=(10, FMAX))

plt.tight_layout()

# ─── SAVE FIGURES ─────────────────────────────────────────────────
root = "/Users/patrick_bloniasz/Documents/Test-FPP/filtered-point-process-test/"
outdir = f"{root}/filtered-point-process/examples/Result_3"
os.makedirs(outdir, exist_ok=True)
fig.savefig(os.path.join(outdir, "Result_3.svg"), format="svg")
fig.savefig(os.path.join(outdir, "Result_3.png"), format="png", dpi=300)

plt.show()